In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
full = pd.read_csv('BorylationTrainingTest 1-10-25.csv')

In [11]:
#group the compounds by numbers
full['grouped'] = full.groupby('Substrate', sort=False).ngroup()
full[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
966,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
967,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
968,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
969,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188


In [12]:
#Convert substrates to Mordred features
full = get_Mordred(full)

100%|██████████| 971/971 [01:30<00:00, 10.78it/s]


In [13]:
final_selected_features = ['Hirshfeld Heavy Atom Charge', 'Hirshfeld Carbon Charge', 'Hirshfeld Hydrogen Charge', 'ESP Heavy Atom Charge', 'ESP Hydrogen Charge', 'NPA Hydrogen Charge', 'Mulliken Heavy Charge', 
                           'Mulliken Hydrogen Charge', 'Steric Effect Index', 'Atomic_Polarizability', 'Distance Degree', 'Dreiding Energy', 'Sigma Electronegativity', 'AATS2d', 'AATS1s', 'AATS2Z', 'AATS0v', 'AATS4p', 'AATS1i', 'ATSC1d', 'ATSC2d', 'AATSC2dv', 'AATSC4d', 'AATSC3Z', 'AATSC5v', 'AATSC1pe', 'MATS1c', 
                           'MATS5s', 'MATS1p', 'GATS3c', 'GATS4c', 'GATS1v', 'GATS2pe', 'BCUTZ-1l', 'RPCG', 'FilterItLogS']

#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('Ligand_validation.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[final_selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(full, "grouped", set_numbers)
    test_data = filter_rows_by_values2(full, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[final_selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

Val_results

100%|██████████| 16/16 [00:01<00:00, 13.95it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,1,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1
5,0,1,1,0,1,1,1,1,1,1,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1


In [14]:
# Read Training/Test data input File
ligand = pd.read_csv('BorylationTrainingTest_ligand 1-10-25.csv')

In [15]:
#group the compounds by numbers
ligand['grouped'] = ligand.groupby('Substrate', sort=False).ngroup()
ligand[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
1026,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
1027,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
1028,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
1029,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188


In [16]:
#Convert substrates to Mordred features
ligand = get_Mordred(ligand)

100%|██████████| 1031/1031 [01:27<00:00, 11.81it/s]


In [17]:
ligResults_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

ligmaxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(ligand, "grouped", set_numbers)
    test_data = filter_rows_by_values2(ligand, "grouped", set_numbers)
  
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop(['Product_Ratio','grouped'], axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop(['Product_Ratio','grouped'], axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred_rfc = rfc.predict(test_X)
        ligResults_df = classificationMetrics(ligResults_df, test_y, pred_rfc)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = ligResults_df.groupby('y')['acc'].mean()
        acc_std = ligResults_df.groupby('y')['acc'].std()
        precision_0_mean = ligResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = ligResults_df.groupby('y')['precision 0'].std()
        precision_1_mean = ligResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = ligResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = ligResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = ligResults_df.groupby('y')['recall 0'].std()
        recall_1_mean = ligResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = ligResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = ligResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = ligResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = ligResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = ligResults_df.groupby('y')['F1 1'].std()
        true_neg_mean = ligResults_df.groupby('y')['True Neg'].mean()
        true_neg_std = ligResults_df.groupby('y')['True Neg'].std()
        false_pos_mean = ligResults_df.groupby('y')['False Pos'].mean()
        false_pos_std = ligResults_df.groupby('y')['False Pos'].std()        
        false_neg_mean = ligResults_df.groupby('y')['False Neg'].mean()
        false_neg_std = ligResults_df.groupby('y')['False Neg'].std()      
        true_pos_mean = ligResults_df.groupby('y')['True Pos'].mean() 
        true_pos_std = ligResults_df.groupby('y')['True Pos'].std()   
 

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)

        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                 


        rfcmaxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        rfcmaxacc_copy  = rfcmaxacc.copy()
        rfcmaxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)

    ligmaxacc_comb = ligmaxacc_comb.append(rfcmaxacc_copy)  

#Write the results onto a CSV file, currently commented out 
ligmaxacc_comb.to_csv("10Runs_lig.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("10Runs_lig.csv", mode="a")

In [18]:
ligmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.45,0.848293,0.009877,0.921886,0.006429,0.596452,0.023655,0.886667,0.009057,0.690000,0.026874,0.903912,0.006413,0.639644,0.022408,146.300000,1.494434,18.700000,1.494434,12.400000,1.074968,27.600000,1.074968,1
0.45,0.830598,0.020140,0.933077,0.012420,0.562010,0.039956,0.849752,0.038945,0.751579,0.066079,0.888762,0.016670,0.639754,0.017098,133.300000,13.412877,23.200000,4.829732,9.750000,2.826100,29.250000,1.860249,2
0.45,0.829712,0.016873,0.920335,0.021112,0.557826,0.034661,0.863875,0.037664,0.686523,0.108632,0.890332,0.013927,0.610764,0.045358,137.933333,12.752101,21.400000,4.724113,12.266667,4.314637,26.733333,3.956081,3
0.45,0.827603,0.015309,0.925099,0.020387,0.557192,0.030271,0.854494,0.036475,0.715517,0.107408,0.887557,0.013098,0.622150,0.044502,134.025000,12.962446,22.475000,4.500641,11.175000,4.217713,28.075000,4.178010,4
0.45,0.835003,0.020469,0.929125,0.020024,0.565568,0.032596,0.860785,0.035061,0.726072,0.098568,0.892968,0.016121,0.632339,0.045214,140.000000,16.725881,22.200000,4.090806,10.840000,3.840493,28.760000,3.992646,5
0.35,0.835787,0.023401,0.925808,0.018408,0.564203,0.043810,0.865763,0.030302,0.710768,0.084387,0.894346,0.017083,0.626676,0.050726,143.350000,15.399565,21.983333,3.993604,11.500000,3.254723,28.333333,3.587486,6
0.35,0.837780,0.022340,0.926067,0.017209,0.571241,0.044445,0.867702,0.028571,0.714358,0.079144,0.895554,0.016165,0.632709,0.049566,141.714286,14.807110,21.428571,3.962035,11.328571,3.063206,28.385714,3.341842,7
0.45,0.833036,0.018735,0.927495,0.016107,0.560459,0.034405,0.859732,0.029784,0.721368,0.078500,0.891872,0.014385,0.628429,0.039261,140.487500,15.013912,22.637500,3.816167,11.075000,3.080256,28.675000,3.165378,8
0.35,0.832615,0.024627,0.925563,0.015721,0.555941,0.050069,0.861788,0.030247,0.710334,0.070722,0.892183,0.017787,0.621731,0.049283,143.288889,15.357350,22.822222,4.540895,11.511111,2.744737,28.266667,2.974801,9


In [19]:
final_selected_features = ['Active Catalyst-Ligand', 'Buried_Vol', 'SASA_area', 'Sterimol_L', 'Hirshfeld Heavy Atom Charge', 'Hirshfeld Carbon Charge', 
                           'Hirshfeld Hydrogen Charge', 'ESP Heavy Atom Charge', 'ESP Hydrogen Charge', 'Mulliken Heavy Charge', 
                           'Mulliken Hydrogen Charge', 'Temp', 'Steric Effect Index', 'Atomic_Polarizability', 'Distance Degree', 
                           'AATSC1pe', 'MATS5Z', 'GATS4d', 'GATS1i', 'BCUTd-1h', 'BCUTse-1l', 'BertzCT']

#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('Ligand_validation.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[final_selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(ligand, "grouped", set_numbers)
    test_data = filter_rows_by_values2(ligand, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_X = training_X.drop('grouped', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_X = test_X.drop('grouped', axis = 1)
    test_y = test_data['Product_Ratio']
    
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[final_selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=15)
    rfc.fit(X_train, y_resampled)
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)


Val_results

100%|██████████| 16/16 [00:01<00:00, 14.63it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,1,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,1


In [20]:
Val_results.to_csv("10Runs_lig.csv", mode="a")

In [21]:
selected_features = ['Active Catalyst-Ligand', 'Buried_Vol', 'SASA_area', 'Sterimol_L', 'Hirshfeld Heavy Atom Charge', 'Hirshfeld Carbon Charge', 
                           'Hirshfeld Hydrogen Charge', 'ESP Heavy Atom Charge', 'ESP Hydrogen Charge', 'Mulliken Heavy Charge', 
                           'Mulliken Hydrogen Charge', 'Temp', 'Steric Effect Index', 'Atomic_Polarizability', 'Distance Degree', 
                           'AATSC1pe', 'MATS5Z', 'GATS4d', 'GATS1i', 'BCUTd-1h', 'BCUTse-1l', 'BertzCT']

#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation1-9-25.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(ligand, "grouped", set_numbers)
    test_data = filter_rows_by_values2(ligand, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 

Val_results.to_csv("10Runs_lig.csv", mode="a")

Val_results

100%|██████████| 81/81 [00:10<00:00,  7.95it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,1,1
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,1
9,0,0,0,0,0,0,0,0,0,0,0
